In [113]:
# Import necessary libraries
import requests 
from io import BytesIO
from PIL import Image
import cv2 as cv
import numpy as np

In [114]:
# Fetch an image from a URL, This line sends an HTTP GET request to the URL
# a status code of 200 in HTTP response means "OK" or "Success."
response = requests.get('https://i.stack.imgur.com/ukOkD.jpg')
response

<Response [200]>

In [115]:
# response.content contains the raw binary content of the image.
# BytesIO is a class in the io module that provides a binary stream interface to an in-memory buffer.
# The Image.open function from the Python Imaging Library (PIL) is used to open the image. 
# The Image.open takes the binary stream created from the image content as an argument, effectively loading the image into memory.
# Image.open from the Pillow library (PIL), the default mode for reading the image is 'RGB'
# The resulting image, which is initially a PIL (Pillow) Image object, is converted to a NumPy array using np.asarray
# np.asarray conversion allows for easier manipulation and processing of the image using NumPy and other libraries like OpenCV.
image = np.asarray(Image.open(BytesIO(response.content)))

In [116]:
# cv.cvtColor is a function in OpenCV used for color space conversion.
# OpenCV typically represents images in BGR order, whereas many other libraries (including PIL/Pillow) use RGB order.
# cv.COLOR_BGR2GRAY: This constant indicates the color space conversion from BGR (Blue, Green, Red) to grayscale.
image_gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

In [117]:
# sobelx defines a 3x3 convolution kernel matrix for horizontal edge detection. 
# sobely defines a 3x3 convolution kernel matrix for vertical edge detection. 
# This specific kernels are known as the Sobel kernels for detecting horizontal and vertical edges in an image.
# In image processing, convolution is a mathematical operation that combines two functions to produce a third.
# convolution involves sliding a small matrix (kernel) over the image and computing the weighted sum of the pixel values at each position.
sobelx = np.array([[-1,0,1],[-2,0,2],[-1,0,1]])
sobely = np.array([[-1,-2,-1],[0,0,0],[1,2,1]])

In [118]:
# cv.filter2D is used for 2D convolution, which involves sliding a kernel matrix over the image and computing the weighted sum of pixel values at each position.
# vertical_edge variable contains the output of the convolution operation.this operation emphasizes the horizontal edges in the image, highlighting areas where there is a significant intensity change from left to right.
# hotizontal_edge variable contains the output of the convolution operation.this operation emphasizes the vertical edges in the image, highlighting areas where there is a significant intensity change from top to bottom.
# second argument is ddepth which is Depth of the output image; if it is set to -1, the output image will have the same depth as the input image.
vertical_edge = cv.filter2D(image_gray, -1, sobelx)
horizontal_edge = cv.filter2D(image_gray, -1, sobely)

In [119]:
# The cv.adaptiveThreshold function calculates the threshold for small regions of the image (defined by the neighborhood size) and applies different thresholds to different regions based on their local characteristics. 
# cv.ADAPTIVE_THRESH_MEAN_C: specifies the method for thresholding. It uses the mean of the neighborhood area.
# cv.THRESH_BINARY: The type of thresholding applied after the adaptive thresholding.
# 11: Size of the pixel neighborhood used to calculate the threshold value. It must be an odd number.
# 2: A constant subtracted from the mean or weighted mean (depending on the adaptive method). This can be used to fine-tune the threshold.
# After this operation, image_adth should contain the resulting binary image after adaptive thresholding. Pixels with intensity values above the threshold will be set to the maximum value (255), and pixels below the threshold will be set to 0.
image_adth = cv.adaptiveThreshold(vertical_edge, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 9,2)

In [120]:
# Median filtering is a type of nonlinear filtering that replaces the pixel value in the center of the neighborhood with the median value of all the pixels in the neighborhood.
# Median filtering operation is effective in reducing salt-and-pepper noise while preserving edges.
# 25: The size of the kernel or the neighborhood window. The kernel size must be a positive odd integer.
# The larger the kernel size, the stronger the smoothing effect, but it may also result in more blurring.
image_blur = cv.medianBlur(image_adth, 25)

In [121]:
# 0: The threshold value used for comparison. In this case, it's set to 0, but since Otsu's method is used (cv.THRESH_OTSU), this value is not critical.
# 255: The maximum pixel value that can be assigned to the output pixels. 
# cv.THRESH_BINARY specifies that pixels with intensities above the threshold will take the value 255, and pixels below the threshold will take the value 0.
# cv.THRESH_OTSU flag indicates that the optimal threshold value should be determined automatically using Otsu's method.
# ret: The computed threshold value. In this case, it's the threshold determined by Otsu's method
# after this operation, image_gray should contain a binary image where the threshold is automatically determined using Otsu's method. 
ret, image_gray = cv.threshold(image_blur, 0, 255, cv.THRESH_BINARY+cv.THRESH_OTSU)


In [122]:
# contour detection function
def function(image_gray,image, horizontal_edge):

    # cv.findContours function in OpenCV finds contours in the grayscale image
    # cv.RETR_TREE: The contour retrieval mode. This mode retrieves all of the contours and reconstructs a full hierarchy of nested contours. Contours are organized as a tree structure.
    # cv.CHAIN_APPROX_SIMPLE: The contour approximation method. This method compresses horizontal, vertical, and diagonal segments and leaves only their end points. For example, an up-right rectangular contour is encoded with only four points.
    # contours: A list of contours found in the image. Each contour is represented as a list of points.
    # _: The hierarchy of contours. In this case, it's not being used, so it is assigned to an underscore to indicate that it is not needed.
    contours, _ = cv.findContours(image_gray, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    
    # Initializes an empty list to store the contours that meet the specified area criteria.
    new_counters = []

    # c represents a contour and i is its index in the list.
    # cv.contourArea(c): Calculates the area of the current contour
    # if statement Checks whether the area of the current contour falls within the specified range
    for i, c in enumerate(contours):
        area = cv.contourArea(c)
        if area>3500 and area<6000:
            new_counters.append(c)
            
    # contours_poly: stores the approximated polygons for each contour, Each element in the list corresponds to a contour, and initially, it is set to None. After processing, it will contain the approximated polygon for the corresponding contour.    
    # boundRect: This list will store the bounding rectangles for each contour.
    # lines: This list will store boolean values indicating whether each contour is considered a "line", all elements are initially set to False. Later, some elements may be set to True
    contours_poly = [None]*len(new_counters)
    boundRect = [None]*len(new_counters)
    lines = [False]*len(new_counters)


    for i, c in enumerate(new_counters):

        # Calculates the perimeter of the contour c.
        epsilon = 0.01 * cv.arcLength(c, True) 

        # Approximates the contour c with a polygon, where epsilon is the approximation accuracy. 
        contours_poly[i] = cv.approxPolyDP(c, epsilon, True)

        # Calculates the bounding rectangle for the approximated polygon 
        boundRect[i] = cv.boundingRect(contours_poly[i])
        
        x, y, width, height = boundRect[i]
        
        # Extracts a region from the horizontal_edge image based on the bounding rectangle.
        extracted_region = horizontal_edge[ y:y+height, x:x+width]
        _, extracted_region = cv.threshold(extracted_region, 0, 255, cv.THRESH_BINARY+cv.THRESH_OTSU)

        # Finds contours in the thresholded extracted region 
        contours, _ = cv.findContours(extracted_region, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

        # Checks the area of each contour (area > 50). If the area is above the specified threshold, it sets the corresponding element in the lines list to True.
        for _, cc in enumerate(contours):
            area = cv.contourArea(cc)
            print(area)
            if area>50:# and area<6000:
                lines[i] = True
                
    for i in range(len(new_counters)):

        # Define color values for red and green.
        red = (0, 0, 255)
        green = (0, 255, 0)

        # Checks whether the current contour is considered a "line" based on the lines list. If it is a line, it draws a green rectangle around the region of interest:
        # These rectangles are drawn on the original image to visually represent the identified regions
        # The green rectangles indicate regions identified as lines, while the red rectangles indicate other regions
        # The cv.rectangle function is used to draw rectangles on the image, and the coordinates of the rectangles are determined by the bounding rectangles (boundRect).
        if lines[i]:
            cv.rectangle(image, (int(boundRect[i][0]), int(boundRect[i][1])), \
            (int(boundRect[i][0]+boundRect[i][2]), int(boundRect[i][1]+boundRect[i][3])), green, 2)

        # If it is not a line, it draws a red rectangle around the region of interest:   
        else:
            cv.rectangle(image, (int(boundRect[i][0]), int(boundRect[i][1])), \
            (int(boundRect[i][0]+boundRect[i][2]), int(boundRect[i][1]+boundRect[i][3])), red, 2)
                
    cv.imshow('Contours', image)

In [123]:
# creating a graphical user interface (GUI) using OpenCV to display an image (image)
# Creates a window with the name 'Source' and show originaal image.
window = 'Source'
cv.namedWindow(window)
cv.imshow(window, image)

# Calls the function
function(image_gray,image,horizontal_edge)

# Waits indefinitely for a key press. The program will continue running until a key is pressed
cv.waitKey(0) 

#  Closes all OpenCV windows.
cv.destroyAllWindows() 

5.0
14.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
3.0
0.5
6.5
4.5
0.0
1.0
30.0
11.0
14.5
0.0
4.5
4.0
1.0
0.0
87.0
0.0
0.5
0.0
0.0
0.0
0.0
3.5
5.5
0.0
22.0
0.0
4.0
17.0
10.5
1.0
5.5
0.0
0.0
1.0
0.0
70.0
0.0
0.0
1.0
0.5
0.0
0.0
0.0
0.0
0.0
23.5
18.5
10.0
0.0
0.0
0.5
4.0
4.0
9.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
37.0
3.0
6.5
2.5
0.0
2.5
0.0
0.0
0.0
77.5
0.5
0.0
0.0
1.5
0.0
1.0
0.0
0.0
0.0
3.0
0.0
32.5
0.0
7.0
11.5
20.0
1.5
2.5
0.0
0.0
0.0
2.0
0.0
6.5
0.0
10.5
0.0
28.0
1.0
0.0
0.0
0.0
0.0
0.0
111.5
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
2.5
5.5
9.0
40.5
52.5
6.5
0.0
0.5
3.5
14.0
0.0
1.0
7.0
1.0
1.0
2.5
0.0
0.0
0.0
2.5
0.0
0.5
2.5
0.0
0.0
2.5
0.0
0.5
5.0
0.0
3.5
4.0
0.0
0.0
0.5
29.5
0.5
2.0
5.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
23.0
1.0
19.5
6.0
51.0
2.5
87.0
2.5
0.0
10.0
113.0
0.0
